In [1]:
import pandas as pd
import requests
import json
from config import api_id_ru, api_key_ru
import time
from tqdm import tqdm

from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
# import re

In [2]:
url = 'https://www.healthline.com/nutrition/20-delicious-high-protein-foods#TOC_TITLE_HDR_2'

response = requests.get(url)

# browser.visit(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [14]:
#for loop tags list 1
tags1 = []

for i in range (1,10):
    want = soup.find_all('div', class_ = "css-0")[i]
    new=want.find('h2')
    tags1.append(new.text)
    
#for loop tags list 2
tags2 = []

for i in range (10,21):
    want = soup.find_all('div', class_ = "css-0")[i]
    new=want.find('h2')
    tags2.append(new.text)

In [15]:
newlst=[]
for i in tags1:
    newlst.append(i[4:])

newlst2=[]
for i in tags2:
    newlst2.append(i[5:])

In [16]:
# Joining lists
ingredients = newlst+newlst2

In [17]:
#Remove (all types) from fish

ingredients=[y.replace(' (all types)', '') for y in ingredients]

In [18]:
# Final list
ingredients

['Eggs',
 'Almonds',
 'Chicken breast',
 'Oats',
 'Cottage cheese',
 'Greek yogurt',
 'Milk',
 'Broccoli',
 'Lean beef',
 'Tuna',
 'Quinoa',
 'Whey protein supplements',
 'Lentils',
 'Ezekiel bread',
 'Pumpkin seeds',
 'Turkey breast',
 'Fish',
 'Shrimp',
 'Brussels sprouts',
 'Peanuts']

In [19]:
# List of ingredients scraped from healthline, top protein sources 
ingredients
# Only interested in recipes where cook time is over 5 minutes
cook_time = '5%2B'
# Total recipes to be queried is 50
from_number = "0"
to_number = "50"

# recipe info lists
recipe_time = list()
recipe_names = list()
recipe_yields = list()
recipe_urls = list()
recipe_main = list()

# This loop pulls 50 recipes per ingredient
for food in tqdm(ingredients):
    url = "https://api.edamam.com/search" + "?q=" + food + "&app_id=" + api_id_ru + "&app_key=" + api_key_ru + "&time=" + cook_time + "&from=" + from_number + "&to=" + to_number
    food_request = requests.get(url)
    food_json = food_request.json()
    
    try:
        # This loop will save recipe information for each ingredient
        for x in range(0, 50):
            # Extract all recipe names for ingredient, make it the key of the dictionary
            name = food_json["hits"][x]["recipe"]["label"]
            recipe_names.append(name)
            # Extract yields for each for ingredient
            yield_amt = food_json["hits"][x]["recipe"]["yield"]
            recipe_yields.append(yield_amt)
            # Extract the url for the recipe
            recipe_url = food_json["hits"][x]["recipe"]["url"]
            recipe_urls.append(recipe_url)
            # Extract the total cook time for the recipe
            recipe_cook_time = food_json["hits"][x]["recipe"]["totalTime"]
            recipe_time.append(recipe_cook_time)
            # Adding Main Ingredient
            recipe_main.append(food)
    
    except IndexError:
        recipe_names.append('N/A')
        recipe_yields.append('N/A')
        recipe_urls.append('N/A')
        recipe_time.append('N/A')
        recipe_main.append('N/A')

    # Sleep in between the queries to avoid getting flagged 5 calls/min
    time.sleep(11)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:02<00:00, 12.13s/it]


In [20]:
# Creating the dataframe for the files.
# List of recipe column names
recipe_columns = ["Recipe Name","Cook Time","URL","Yield","Main Ingredient"]
recipe_df = pd.DataFrame({"Recipe Name" : recipe_names,
                        "Cook Time" :recipe_time,
                        "Yield": recipe_yields,
                        "Main Ingredient": recipe_main,
                        "URL": recipe_urls})

# Creating a dataframe for the coupons
# List of coupon column names
coupon_columns = ["Coupon", "Exp-Date","Amount","URL","Main Ingredient"]
coupon_df = pd.DataFrame(columns = coupon_columns)


In [21]:
recipe_df

,Recipe Name,Cook Time,Yield,Main Ingredient,URL
0,Scotch Eggs Recipe,25,4,Eggs,http://www.seriouseats.com/recipes/2012/03/sco...
1,Moonstruck eggs,24,2,Eggs,https://food52.com/recipes/518-moonstruck-eggs
2,Eggs Benedict,60,4,Eggs,http://www.bbc.co.uk/food/recipes/eggsbenedict...
3,Deviled eggs,17,4,Eggs,http://honestcooking.com/deviled-eggs/
4,Scrambled Eggs recipes,15,2,Eggs,http://www.tastingtable.com/entry_detail/chefs...
...,...,...,...,...,...
940,Thai Cucumber Salad with Peanuts,20,4,Peanuts,http://www.delish.com/cooking/recipe-ideas/rec...
941,Asian Chicken with Peanuts recipes,45,4,Peanuts,http://www.myrecipes.com/recipe/asian-chicken-...
942,"Grilled Shrimp with Cilantro, Lime, and Peanut...",20,4,Peanuts,http://www.marthastewart.com/339717/grilled-sh...
943,Diane's Sugared Peanuts,50,4,Peanuts,https://www.foodnetwork.com/recipes/paula-deen...


In [25]:
new_recipe_df = recipe_df.loc[(recipe_df["Recipe Name"] != "N/A")]

In [26]:
new_recipe_df

,Recipe Name,Cook Time,Yield,Main Ingredient,URL
0,Scotch Eggs Recipe,25,4,Eggs,http://www.seriouseats.com/recipes/2012/03/sco...
1,Moonstruck eggs,24,2,Eggs,https://food52.com/recipes/518-moonstruck-eggs
2,Eggs Benedict,60,4,Eggs,http://www.bbc.co.uk/food/recipes/eggsbenedict...
3,Deviled eggs,17,4,Eggs,http://honestcooking.com/deviled-eggs/
4,Scrambled Eggs recipes,15,2,Eggs,http://www.tastingtable.com/entry_detail/chefs...
...,...,...,...,...,...
940,Thai Cucumber Salad with Peanuts,20,4,Peanuts,http://www.delish.com/cooking/recipe-ideas/rec...
941,Asian Chicken with Peanuts recipes,45,4,Peanuts,http://www.myrecipes.com/recipe/asian-chicken-...
942,"Grilled Shrimp with Cilantro, Lime, and Peanut...",20,4,Peanuts,http://www.marthastewart.com/339717/grilled-sh...
943,Diane's Sugared Peanuts,50,4,Peanuts,https://www.foodnetwork.com/recipes/paula-deen...


In [27]:
new_recipe_df.loc[(recipe_df["Main Ingredient"] == 'Eggs')]

,Recipe Name,Cook Time,Yield,Main Ingredient,URL
0,Scotch Eggs Recipe,25,4,Eggs,http://www.seriouseats.com/recipes/2012/03/sco...
1,Moonstruck eggs,24,2,Eggs,https://food52.com/recipes/518-moonstruck-eggs
2,Eggs Benedict,60,4,Eggs,http://www.bbc.co.uk/food/recipes/eggsbenedict...
3,Deviled eggs,17,4,Eggs,http://honestcooking.com/deviled-eggs/
4,Scrambled Eggs recipes,15,2,Eggs,http://www.tastingtable.com/entry_detail/chefs...
5,Devilled Eggs,30,8,Eggs,http://www.cookstr.com/recipes/devilled-eggs-2
6,Deviled Eggs,131,24,Eggs,http://www.marthastewart.com/354137/deviled-eggs
7,Eggs Benedict recipes,25,4,Eggs,http://chezus.com/2014/04/07/eggs-benedict/
8,Deviled Eggs,45,12,Eggs,https://www.realsimple.com/food-recipes/browse...
9,Poached Eggs,5,4,Eggs,http://www.delish.com/cooking/recipe-ideas/rec...


In [28]:
new_recipe_df.to_csv('../../datasets/data.csv')

In [23]:
print(len(recipe_names))
print(len(recipe_time))
print(len(recipe_yields))
print(len(recipe_urls))
print(len(recipe_main))

945
945
945
945
945
